# Invoice reading NLP system
Remember this image? IT IS BACK!!!
![System image](https://storage.googleapis.com/aibootcamp/general_assets/ml_system_architecture.png)


This week is all about system building. Because hardly ever does a ML system stand alone. Your success in building a system for Ortec Finance depends as much on what is around your neural net as it depends on the neural net itself. This baseline is my approach to the problem. Much in this notebook was hacked together so I am sure you can improve on many points. Perhaps you even come up with a completely different approach.

## The approach, character wise classification:
The goal of the task is to extract information from the invoice. The invoice has been run through optical character recognition (OCR). OCR turns PDFs into texts but often messes up the order and confuses come characters. **To extract information from this text, we classify each character by category**. 

Take an example, if we just wanted to get the amount we would classify the characters like this:

|T|O|T|A|L|:| |€| |4|3|6|.|0|0|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|0|0|0|0|0|0|0|1|1|1|1|1|1|1|1|

We classify our text into 6 classes here:

Ignore:           0
Sender Name:      1 
Sender KVK:       2 
Sender IBAN:      3 
Invoice Reference:4
Total:            5

These are the classes that the training data generator tags. But the class of a character does not only depend on the character. It depends on its surroundings as well. To train our model, we create substrings of our invoice that include a certain amount of preceeding and succeeding characters. The amount of preceding and succeeding characters is defined in the `PADDING` global variable. 

If for example we wanted to classify the character '€' from the example above and had `PADDING = 3` we would feed
'L: € 43' into our network. You can see how the amount of padding has a great influence on the performance of our system.

## Post processing:
A significant part of model performance stems from what is done with the outputs of the neural net. This approach groups predictions to prediction sequences and only keeps predictions in which 5 consecutive characters were grouped into the same category. An approach to try would be to allow sequences to be interrupted by one character. Another nice add on would be to rank predicted sequences by the total confidence the neural network has in the sequence. 

## Some tips:
For this assignment you can dive pretty deep into software development. 
You might find these jupyter tricks helpful: https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

Especially debugging with `pdb` really makes things easier: https://docs.python.org/3.5/library/pdb.html#debugger-commands

Basically, if anything crashes, you can start a new cell and enter `%debug`. You then come to a command line in which you can look around what happened at the crash.
The debugger has some special commands. For example `p my_var` prints out a variable. This also works for other python operations, e.g. `p len(my_list)`.

Good luck with building a great system!

In [25]:
!git clone https://github.com/riklmr/MLiFC_data_invoices

fatal: destination path 'MLiFC_data_invoices' already exists and is not an empty directory.


In [28]:
!cd MLiFC_data_invoices ;git pull

remote: Counting objects: 3, done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/riklmr/MLiFC_data_invoices
   42cd6ea..1adf5e3  master     -> origin/master
Updating 42cd6ea..1adf5e3
Fast-forward
 invoices_train.z01     | Bin 52428800 -> 0 bytes
 invoices_train.z02     | Bin 52428792 -> 0 bytes
 invoices_train.zip     | Bin 1666554 -> 0 bytes
 invoices_train_20k.zip | Bin 0 -> 21303034 bytes
 4 files changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 invoices_train.z01
 delete mode 100644 invoices_train.z02
 delete mode 100644 invoices_train.zip
 create mode 100644 invoices_train_20k.zip


In [0]:
!cd MLiFC_data_invoices

In [30]:
!ls MLiFC_data_invoices

invoices_train_20k.zip	LICENSE  README.md


In [31]:
!unzip MLiFC_data_invoices/invoices_train_20k.zip -q

Archive:  MLiFC_data_invoices/invoices_train_20k.zip
   creating: train/
  inflating: train/09562215-ec1e-42d6-9526-fc3adf37eb68.csv  
  inflating: train/6814aa63-4d54-4f08-b763-91a90ea64afd.csv  
  inflating: train/3c9e0551-b35c-4e33-8dd8-a63f246e9899.csv  
  inflating: train/1502888c-d11a-4823-b424-bd82b3d9fd6e.csv  
  inflating: train/5889b787-625a-463e-b7b3-2826d8d3d06e.csv  
  inflating: train/78afe0ca-21ed-42bb-8d94-dc2316dd02a9.csv  
  inflating: train/9ce2d52a-9b21-4811-940f-bda76e513427.csv  
  inflating: train/1fae3397-8e53-454d-8a8f-464a67bc170d.csv  
  inflating: train/a6578490-6e7e-428a-afb1-ebb0fecad000.csv  
  inflating: train/3860113d-6038-4436-8431-da69f4f75e26.csv  
  inflating: train/56c5727c-7b9d-4e74-b716-b51dc6a07e41.csv  
  inflating: train/58657bb9-696b-4c8d-be5e-9ceb8e27fd09.csv  
  inflating: train/63a690ea-69fd-4ee9-851b-0cf6295f8c14.csv  
  inflating: train/f88d2c9c-cf7d-45fa-ac13-b769f4a6d23c.csv  
  inflating: train/bb679bdc-5c22-4fb5-ba35-a122436e24a1.csv

  inflating: train/3ac1c461-3043-44a5-906c-1a80eda9b163.csv  
  inflating: train/366373b2-915a-4264-b4aa-aef7d1eeb2a5.csv  
  inflating: train/59f6b11f-c363-4470-afeb-d5bf7cc28f57.csv  
  inflating: train/e9946e0e-dc3a-476c-ab31-c4de36559a19.csv  
  inflating: train/a48f5584-ee97-4922-9d58-052f07163f89.csv  
  inflating: train/f7ac956f-40a3-486b-9cd0-102da1fbab68.csv  
  inflating: train/faeccd64-599e-4d30-a523-8d2584fbc1ce.csv  
  inflating: train/85361cb7-6ec0-4a49-ae5d-d681261e50bb.csv  
  inflating: train/57c961a4-8c0e-4c0d-91c3-b6d55fbdefbe.csv  
  inflating: train/b04d0b2e-d56e-4305-9575-df31eb1911b0.csv  
  inflating: train/cb4a8df5-a27b-4101-b4e6-9dc737653379.csv  
  inflating: train/b4c431c5-f705-4f41-88c9-e8018a7cf3fb.csv  
  inflating: train/27b8cfb5-a4fb-4c83-a1a8-e748e3054e64.csv  
  inflating: train/a44e45f1-5c4b-484e-b8e5-cc2f11dda561.csv  
  inflating: train/8d00d3d0-6f7a-4e37-b0d9-e7981ca31bf4.csv  
  inflating: train/6bf4d9e9-8337-4376-ab0c-c0228843c77a.csv  
  inflat


  inflating: train/bbc8ae90-3343-486a-af00-7a61277be493.csv  
  inflating: train/6a1e9b67-2301-4c8f-99b5-66317540b2cf.csv  
  inflating: train/0d800a5a-dabc-4d0c-be51-96d2d36a11e5.csv  
  inflating: train/f1d86321-be1c-4c6d-8045-82e52a657cd7.csv  
  inflating: train/6a777838-27af-4768-8bdb-e42069b1ccce.csv  
  inflating: train/aafbcc81-fe48-486b-a189-57cf84d16574.csv  
  inflating: train/72f36908-d33a-4aa1-bdfe-f34f13ba7ff1.csv  
  inflating: train/ddd50f2d-e72a-44fd-b13c-4c2dc19797b1.csv  
  inflating: train/ac166851-cd9a-4f96-afef-77f1d254e289.csv  
  inflating: train/2e249cee-0f36-40ca-8a00-8cd15afd7560.csv  
  inflating: train/48f8d83e-9813-43e7-b67e-0afd139a81d8.csv  
  inflating: train/8ea9e52f-909d-4f5b-b4b4-28471f44329d.csv  
  inflating: train/d7ad9301-7b8f-4475-9583-95ac2b0bd608.csv  
  inflating: train/f2350091-19be-4089-b602-3321b0e4a950.csv  
  inflating: train/2b1cd39b-d310-4784-be64-c438c4aa8f18.csv  
  inflating: train/61500b57-b628-40ba-a65b-eab2f5d50599.csv  
  infla


  inflating: train/db9c7bde-adcc-4b3b-9614-21728af7feb4.csv  
  inflating: train/8bf803d4-3e2d-4b89-8672-bfc08708e9ce.csv  
  inflating: train/3bc6f647-7c14-488e-8c85-784bdddcae52.csv  
  inflating: train/9e6eb18b-f26d-40d9-ad7b-29627921134a.csv  
  inflating: train/c7ec2721-52c9-4e29-801f-5f815ef3bf04.csv  
  inflating: train/7d7793a0-2bc0-4875-aa25-a25cb0624fd4.csv  
  inflating: train/c48186c0-fd45-4861-9970-f41d0f96fa59.csv  
  inflating: train/5b33f55f-71a5-4bef-b32c-06565509a793.csv  
  inflating: train/7577d0a6-317e-472a-8fef-745c662c0596.csv  
  inflating: train/23948949-c5c1-436a-bbec-8d7addf86dd1.csv  
  inflating: train/96fea830-5770-4524-8061-47ce30c0995c.csv  
  inflating: train/abb2ec38-877f-4841-9e30-eee7f4208254.csv  
  inflating: train/bc0bb2b3-41e6-4e14-be9d-86d756d19b33.csv  
  inflating: train/646fdcc0-f77d-40c6-8f31-71bf2cadcaa7.csv  
  inflating: train/dad7adab-f9e6-4020-aad3-8a24b397c282.csv  
  inflating: train/027d2d6d-f3ad-4cb4-8737-c439568dda5f.csv  
  infla

  inflating: train/e8d8baf3-30c5-4399-82ab-398876eda662.csv  
  inflating: train/ff82d219-887c-4381-9c22-b1db8dadae2d.csv  
  inflating: train/c190a9c5-18ca-4ab3-b329-a5eb85b8ba83.csv  
  inflating: train/1cf12820-610d-44bf-b9ce-d8fd1cadd3d6.csv  
  inflating: train/05d1f9b5-35b3-4d79-953d-a27a02a3a35d.csv  
  inflating: train/81aab053-27c4-47d4-b3ce-a2b59478dae4.csv  
  inflating: train/a7e9bc4c-6891-420e-94ca-b96731dc138c.csv  
  inflating: train/1bbdde26-a657-4bbb-9cc3-6db7f7515458.csv  
  inflating: train/6e83e196-1b61-47e3-97c6-3484c794c35a.csv  
  inflating: train/198baa6f-692e-4073-bb36-f9271554f418.csv  
  inflating: train/abff84ab-42b4-4db0-b082-134b41817cf9.csv  
  inflating: train/333d94d7-6caf-45ad-974c-5655a5b4e3e8.csv  
  inflating: train/b82ad212-be2e-4e8e-aaad-e8f4cd2edb70.csv  
  inflating: train/8743d6ca-5b52-40b3-9ded-f62321fa1162.csv  
  inflating: train/645d1cf0-73a4-4213-ae1a-d3611069b14c.csv  
  inflating: train/dceb589f-b69c-4a1d-9245-c346593310e1.csv  
  inflat

...


  inflating: train/69298edb-9d22-4ff2-939e-ef3e3c8b9b71.csv  
  inflating: train/30db0756-7a4b-4db1-a07f-0ccc573f28b5.csv  
  inflating: train/36bbe351-ec10-4eeb-99fe-107d0abada76.csv  
  inflating: train/cef19cbd-0dbb-4f1e-99ab-a280d5395ee4.csv  
  inflating: train/232bea43-cf25-43bc-92ca-d988dd1b84a4.csv  
  inflating: train/5179d39e-ccc0-4abc-a2e6-9caae8095f91.csv  
  inflating: train/c3e0d7ea-e8e2-4d22-8304-97b818b3b627.csv  
  inflating: train/9e638e67-65c9-45b8-b0ea-07e7ea21dd2b.csv  
  inflating: train/5fb7c4c2-4be4-4c4a-9de1-bf1034cdc030.csv  
  inflating: train/022b0bb3-c098-4c91-bf46-e4fd56d19fd1.csv  
  inflating: train/ec67af2e-ff53-4f55-bc55-d2595c3f4105.csv  
  inflating: train/d7dc79cf-3cb1-4c65-9ff3-9f05084e981e.csv  
  inflating: train/cdf28240-c0b1-45aa-949b-177d2adad326.csv  
  inflating: train/6faf43b2-88a1-49e5-b98a-be7ba34a1736.csv  
  inflating: train/b035d705-4fed-410b-8509-7e8676d1723c.csv  
  inflating: train/f9b071c2-07b0-42bd-8afd-0ff9e5483269.csv  
  infla

  inflating: train/721de94f-3a57-4d95-bd17-c15fcafaa4c1.csv  
  inflating: train/026bd0d0-3873-4353-b958-b9887da3efef.csv  
  inflating: train/66cee367-3942-4f93-9734-12985fb8cdfa.csv  
  inflating: train/b5d86e61-826c-4c33-becc-e6018ce4d9b1.csv  
  inflating: train/ae76fefd-4feb-49de-a283-913d5a153e4b.csv  
  inflating: train/e6b3f12b-cc77-45bb-96ef-77c2d8f75181.csv  
  inflating: train/10525a56-b1a9-448c-beca-ed89ef1fd429.csv  
  inflating: train/d3060dac-dfa4-4784-97a9-db7bc7fca095.csv  
  inflating: train/d7750148-3068-4c55-954a-7f93c2061c88.csv  
  inflating: train/c0c8f5a6-748b-4f09-815b-d49903d8f898.csv  
  inflating: train/1535ad91-6646-4596-8e01-5c139bb7907e.csv  
  inflating: train/66170221-cce1-446d-870f-b9600b919697.csv  
  inflating: train/817eb588-3f21-435d-a1e2-916f0274b69d.csv  
  inflating: train/ff25bc94-1de0-4128-8e2d-17013b443455.csv  
  inflating: train/f87b111e-096a-41fa-94bf-2961d99f4266.csv  
  inflating: train/fce86d50-e777-479b-bce9-ca7f49005ed3.csv  
  inflat


  inflating: train/e8a5ec87-57b1-4080-aef5-b315687839a8.csv  
  inflating: train/38102127-4408-4bb0-acd1-a3ad69c6f1dd.csv  
  inflating: train/492e0139-26e6-4795-bee5-48dea5538240.csv  
  inflating: train/19ac6c0a-5f6b-463e-b3c0-5725b844d86a.csv  
  inflating: train/b4182c86-db0a-4bbc-8238-3d8513fef398.csv  
  inflating: train/3b461048-bce1-4787-9f17-436bc2a97614.csv  
  inflating: train/34de8a89-7fcf-4d34-9232-000cc07c8572.csv  
  inflating: train/2be74541-5023-4387-9961-f7121e3f9871.csv  
  inflating: train/82ad454f-fa36-4b66-b6aa-ee014dbbec4b.csv  
  inflating: train/df06d367-5fe2-4578-b3e1-5ba612efea87.csv  
  inflating: train/794c1154-ebf6-4bb4-9697-375aa5a49a02.csv  
  inflating: train/abb7f298-70d9-41d2-a194-ac2003e10bae.csv  
  inflating: train/175ea9d2-ab83-41d2-9108-a6a6dc5aafd4.csv  
  inflating: train/f4980a84-a746-4f43-842d-204babdb768e.csv  
  inflating: train/106597d5-1fbb-4b13-9b32-2a7856c5313a.csv  
  inflating: train/1420ef21-e47a-42c9-80e7-a62b2150ae9b.csv  
  infla

  inflating: train/98f54655-b981-48ff-b654-b7efad6a9a2f.csv  
  inflating: train/634ad884-ac22-4317-8a1c-27563c7173eb.csv  
  inflating: train/14ed32d0-81a4-47e8-93c8-42514fc69a5f.csv  
  inflating: train/2114a3ea-bd6f-473d-8df6-2d0633ca1f65.csv  
  inflating: train/a0ef3a3c-de99-4923-ae97-47dd8ff3a904.csv  
  inflating: train/ccf43b81-d5ff-46d5-8f84-51071a26933f.csv  
  inflating: train/2157089e-70e7-440f-a680-549dbf0799e0.csv  
  inflating: train/19c95431-30ec-421b-998d-19de3a729bac.csv  
  inflating: train/5eae8fc8-7530-4d5c-ba15-aa99cc355ee0.csv  
  inflating: train/d0d86bb8-65d8-4885-81be-7743734f16c6.csv  
  inflating: train/342e629b-2517-4fc8-b88f-171be2d545e1.csv  
  inflating: train/393c0ac9-8143-4b3f-b352-58c2349a7641.csv  
  inflating: train/1064e80b-868a-4691-abad-a520aec31be7.csv  
  inflating: train/cd890c57-13ee-40b0-9b11-c3047e787678.csv  
  inflating: train/8e8392dc-5da3-413b-8f5b-20db79511f6d.csv  
  inflating: train/868f3d90-f484-420a-aafa-fe9018cacb14.csv  
  inflat


  inflating: train/fc1f405f-b0c9-4f6d-ba4f-1c3d587ebb52.csv  
  inflating: train/85d4d1a4-33ab-4fa2-b810-2d9df48c9142.csv  
  inflating: train/54adf56a-7895-4b63-bed0-e619bd1c6f25.csv  
  inflating: train/1b6f49dd-f02a-458c-af98-080ca7acfd92.csv  
  inflating: train/a35e474a-cd30-457f-a345-686997e3a255.csv  
  inflating: train/559ee692-2185-4953-b697-fada248e6d3f.csv  
  inflating: train/d131950d-6840-454d-a3c4-970d35fac54b.csv  
  inflating: train/bd74746b-76d3-468a-9ac0-33bb55340897.csv  
  inflating: train/af72fe2f-e024-47fe-a529-902102ec3f4a.csv  
  inflating: train/4ee330a9-31cb-4a1a-af59-70ec25c7bbbd.csv  
  inflating: train/dd11cb21-4dd7-4a51-9bee-f9b632ccfce4.csv  
  inflating: train/1a7389c0-bb7a-4b73-a7ee-abead641c425.csv  
  inflating: train/e7893993-35ef-4196-af1f-9032eb711dd3.csv  
  inflating: train/a11a7cac-449a-4586-a709-d1ca2c936960.csv  
  inflating: train/d592f2ee-fa9d-4d1c-8e3b-3ddb561f71c0.csv  
  inflating: train/e7a503b6-69d9-4fae-b418-7d8cf4172fc6.csv  
  infla

In [34]:
# Shows the disk usage of the directory train 
!du -sh train

134M	train


In [33]:
train.shape

NameError: ignored

In [18]:
!rm -r

datalab  MLiFC_data_invoices


In [0]:
!pip install -q keras

## Loading templates

In [0]:
# System hyper parameters here

# How many characters before and after the main char to feed the NN
PADDING = 20 


'''
Ignore:           0
Sender Name:      1 
Sender KVK:       2 
Sender IBAN:      3 
Invoice Reference:4
Total:            5
'''
N_CLASSES = 6

In [0]:
# Invoice data generator
from templates.invoicegen import create_invoice

In [0]:

# Your friendly tokenizer
from keras.preprocessing.text import Tokenizer

# Numpy
import numpy as np

import pandas as pd

# glob
import glob

In [0]:
# This is the ortec package made by Rik. It should be imported as a package in the final model in GitHub.

def select_batch(start=0, batch_size=32, stop=32):
    while start < stop:
        yield [start, min(stop, start + batch_size)]
        start += batch_size
#

def select_filebatch(filenames=[], start=0, batch_size=32, total=32):
    stop = min(len(filenames)+1, total)
    for (first_idx, last_idx) in select_batch(start=0, batch_size=batch_size, stop=stop):
        yield filenames[first_idx:last_idx]
#

def select_invoicebatch(filenames=[], batch_size=32, total=32):
    """yields list of invoices, list of targets, list of truths"""
    for file_batch in select_filebatch(filenames=filenames, batch_size=batch_size, total=total):
        invoices = []
        targets = []
        truths = []

        for file in file_batch:
            mysample = pd.read_csv(file)
            # each file only contains one row, that's why we get away with the 0 in .loc[0,'invoice']
            # else we needed to start another level of iteration
            invoice  = mysample.loc[0,'invoice']
            target   = eval(mysample.loc[0,'target'])
            truth    = eval(mysample.loc[0,'truth'])
            invoices.append(invoice)
            targets.append(target)
            truths.append(truth)
        #
        yield invoices, targets, truths
    #
#

In [41]:
train_dir = 'train/'
filenames_all = glob.glob(train_dir + "*.csv")
print("{} files found in directory".format(len(filenames_all)))

batch_size = 32


19992 files found in directory


In [49]:
# after importing ortec package, use = ortec.select_invoicebatch...
# for instance, in a for-loop:
workbatch = select_invoicebatch(filenames=filenames_all, batch_size=batch_size, total=70)  

piece = slice(180,270)
for invoice, target, truth in workbatch:
    print()
    print("   retrieved batch of {} invoices".format(len(invoice)))
    # print some of the retrieved content, just for the first invoice in the batch
    print((invoice[0][piece]).replace("\n", " ")) #print a piece of the invoice, replace newlines
    print("".join([str(x) for x in target[0][piece]]) ) #print corresponding piece of target
    print(truth[0][1])  # print the True Sender Name


   retrieved batch of 32 invoices
69393 000019267231 ING NETHERLANDS NL02INGB0681309748 INGBNL2A 06-88163931 info@unilever.c
222220000000000000000000000000000003333333333333333330000000000000000000000000000000000000
Unilever Nederland

   retrieved batch of 32 invoices
 Boompjes 40 3011XB Rotterdam  Invoice Summary Invoice Number: Invoice Date:  QPHNASEKUX 1
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ING Bank N.V.

   retrieved batch of 6 invoices
bject': ''} Oostplein 97 3011 KW Rotterdam (+31) (0)10 7982088 (+31) (0)10 2125351 klanten
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ORTEC Finance B.V.


## This is where we got stuck on how to feed the self-generated invoices to the model. We could only make the originally generated invoices to work. All code after this is not currently working. 

In [54]:
# The all_invoices is a generator object yielding three lists of invoices, targets and truths
all_invoices = select_invoicebatch(filenames=filenames_all)
all_invoices()

for invoice, target, truth in all_invoices:
    pass

AttributeError: ignored

## Generate substring

In [0]:
# Create our tokenizer
# We will tokenize on character level!
# We will NOT remove any characters
tokenizer = Tokenizer(char_level=True, filters=None)
tokenizer.fit_on_texts(invoices)

In [0]:
def gen_sub(inv,tar,pad, m = None):
    '''
    Generates a substring from invoice inv and target list tar 
    using the character at index m as a midpoint.
    
    Params:
    inv - an invoice string
    tar - a target list specifying the type of each item
    pad - the amount of padding to attach before and after the focus character
    
    Returns:
    sub - a string with pad characters, the focus character, pad characters
    '''
    # If no focus character index is set, choose at random
    if m == None:
        m = np.random.randint(0,len(inv))
        
    l = m - pad # define the lower bound of our substring
    h = m + pad + 1 # define the upper (high) of our substring

    # Sometimes, our lower bound could be below zero
    # In this case we attach the remaining characters from the back of the string
    if l < 0:
        # Get the characters from the back of the file
        s1 = inv[l:None]
        
        # Edge case: Sample size larger than string
        # Our upper bound might be higher than the lenth of the text
        # In that case we start from the front again
        if h >= len(inv): 
            # How many characters do we need from the front
            overlap = h - len(inv)
            # The string is the entire invoice + some chars from the front
            s2 = inv
            s_over = inv[None:overlap]
            s2 = s2 + s_over
        else:
            # If we don't need chars from the front 
            # we can just select to the upper bound
            s2 = inv[None:h]
            
        # Create substring
        sub = s1 + s2
        # Ensure the substring has the right length
        assert(len(sub) == pad*2 +1)
        return sub, tar[m]
    
    # Our lower bound might be positive but our upper bound might 
    # still be above the length of the invoice
    elif h >= len(inv):
        # Calc how many chars we need from the front
        overlap = h - len(inv)
        
        # Get string from lower bound to end
        s1 = inv[l:None]
        # Get string from the front of the doc
        s2 = inv[None:overlap]
        sub = s1 + s2
        # Make sure our string has the correct length
        assert(len(sub) == pad*2 +1)
        return sub, tar[m]
    
    # Upper and lower bound lie within the length of the invoice
    else: 
        sub = inv[l:h]
        assert(len(sub) == pad*2 +1)
        return sub, tar[m]

## Generate dataset for training

In [0]:
def gen_dataset(sample_size, n_classes, invoices, targets, tokenizer):
    '''
    Generate a dataset of inputs and outputs for our neural network
    
    Params:
    sample_size - desired sample size
    n_classes - number of classes
    invoices - list of invoices to sample from
    targets - list of corresonding targets to sample from
    tokenizer - a keras tokenizer fit on the invoices
    
    The function creates balanced samples by randomly sampling untill 
    an equal amount of samples of all types is created.
    
    Characters are one hot encoded
    
    Returns:
    x_arr: a numpy array of shape (sample_size, seqence length, number of unique characters)
    y_arr: a numpy array of shape (sample_size,)
    '''
    
    # Create a budget
    budget = [sample_size / n_classes] * n_classes
    
    # Setup holding variables
    X_train = []
    y_train = []

    # While there is still a budget left...
    while sum(budget) > 0:
        # ... get a random invoice and target list
        index = np.random.randint(0,len(invoices))
        inv = invoices[index]
        tar = targets[index]
        # ... sample up to 10 items from this invoice 
        for j in range(10):
            # Get an item
            x, y = gen_sub(inv,tar,PADDING)
            # if we still have a budget for this items target
            if budget[y] > 0:
                # Tokenize to one hot
                xm = tokenizer.texts_to_matrix(x)
                # Add data and target
                X_train.append(xm)
                y_train.append(y)
                budget[y] -= 1
      
    # Create numpy arrays from all data and targets
    x_arr = np.array(X_train)
    y_arr = np.array(y_train)
    return x_arr,y_arr

In [0]:
# Ger data
train_size = 12000

# Shaffy: Increased validation size to 1200
val_size = 1200

x_tr, y_tr = gen_dataset(train_size, N_CLASSES, invoices, targets, tokenizer)
x_val, y_val = gen_dataset(val_size, N_CLASSES, invoices, targets, tokenizer)

In [0]:
x_tr.shape

(12000, 41, 85)

## Model building

In [0]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense,Activation, Conv1D, MaxPool1D

In [0]:
# A simple model
model = Sequential()

# Shaffy: Played around with convolutions and size, but this seems best
model.add(Conv1D(32,2,input_shape=(None, 85))) # The input shape assumes there is 85 possible characters
model.add(MaxPool1D(1))

# Shaffy: Changed layers from 10 to 50
model.add(SimpleRNN(50))
model.add(Dense(6))

# Shaffy: tried a few different activations but this one was best
model.add(Activation('softmax'))

In [0]:
# sparse_categorical_crossentropy is like categorical crossentropy but without converting targets to one hot
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['acc'])

In [0]:
model.fit(x_tr,y_tr,batch_size=32,epochs=6,validation_data=(x_val,y_val))

Train on 12000 samples, validate on 1200 samples
Epoch 1/6
12000/12000 [==============================] - 14s - loss: 0.3681 - acc: 0.9042 - val_loss: 0.1447 - val_acc: 0.9683
Epoch 2/6
12000/12000 [==============================] - 14s - loss: 0.0996 - acc: 0.9749 - val_loss: 0.0860 - val_acc: 0.9767
Epoch 3/6
12000/12000 [==============================] - 14s - loss: 0.0563 - acc: 0.9878 - val_loss: 0.0581 - val_acc: 0.9833
Epoch 4/6
12000/12000 [==============================] - 14s - loss: 0.0490 - acc: 0.9883 - val_loss: 0.0757 - val_acc: 0.9850
Epoch 5/6
12000/12000 [==============================] - 13s - loss: 0.0505 - acc: 0.9875 - val_loss: 0.0483 - val_acc: 0.9867
Epoch 6/6
12000/12000 [==============================] - 13s - loss: 0.0294 - acc: 0.9930 - val_loss: 0.0342 - val_acc: 0.9925


## Generate demo invoice

In [0]:
'''
To make predictions from our model, we need to create 
sequences around every character from the invoice.

We the making predictions for every charater based on their invoice
'''

# Choose a random invoice:
index = np.random.randint(0,len(invoices))
inv = invoices[index]
tar = targets[index]


chars = [] # Holds the individual characters
data = [] # Holds the sequences around the characters
y_true = [] # Holds the true targets for each character

# Loop over characters indices
for i in range(len(inv) -1):
    # Create sequence around this character
    x,y = gen_sub(inv,tar,PADDING,m=i)
    # Tokenize the sequence to one hot
    xm = tokenizer.texts_to_matrix(x)
    # Get the character itself
    c = inv[i]
    
    chars.append(c)
    data.append(xm)
    y_true.append(y)

In [0]:
import pandas as pd

In [0]:
# For demo purposes we can look what our invoice looks like
df = pd.DataFrame({'Char':chars,'Target':y_true})

In [0]:
# Show all characters belonging to the amount
df[df.Target == 5]

,Char,Target
307,1,5
308,7,5
309,8,5
310,1,5
311,7,5
312,.,5
313,4,5
314,9,5


In [0]:
# Create test data for predictions with neural net
x_test = np.array(data)

In [0]:
x_test.shape

(1837, 41, 85)

## Making predictions

In [0]:
# Make predictions
y_pred = model.predict(x_test)

In [0]:
# Get the maximum likely class
y_pred = y_pred.argmax(axis=1)

In [0]:
# Show how our model predictions look like
df['Predicted'] = y_pred

In [0]:
# Show all chars that are predicted to belong to the amount
df[df.Predicted == 5]

,Char,Target,Predicted
305,\n,0,5
306,\n,0,5
307,1,5,5
308,7,5,5
309,8,5,5
310,1,5,5
311,7,5,5
312,.,5,5
313,4,5,5
314,9,5,5


## Obtain system outputs from predictions

In [0]:
from itertools import groupby
# Create groups by the predicted output
# The this code will return a tuple with the format
# (category, length, starting index)

# TODO: This code is ugly and very hard to understand
# But it works

# Group by predicted category
g = groupby(enumerate(y_pred), lambda x:x[1])

# Create list of groups
l = [(x[0], list(x[1])) for x in g]

# Create list with tuples of groups
groups = [(x[0], len(x[1]), x[1][0][0]) for x in l]

In [0]:
# Show grouping
groups[:10]

[(0, 130, 0),
 (4, 6, 130),
 (0, 19, 136),
 (1, 20, 155),
 (0, 125, 175),
 (2, 5, 300),
 (5, 12, 305),
 (0, 601, 317),
 (4, 2, 918),
 (0, 478, 920)]

In [0]:
'''
We only want to consider sequences of predictions of the same type 
that have a minimum length. This way we remove the noise
But we also might remove some good predictions

The min length is set to 5 here, certainly a value to experiment with
'''
candidates = []
# Loop over all groups
for group in groups:
    
    # Unpack group
    category, length, index = group
    
    
     # Shaffy: Raised length from 5 
    if category != 0 and length > 5:
        # Create text
        candidate_text = ''.join(chars[index:index+length])
        
        # Remove line breaks, this is just one way to prettify outputs!
        candidate_text = candidate_text.replace('\n','')
        candidates.append((candidate_text,category))
        
    # Shaffy: we can use these types of arguments similar to Regex    
    if category == 2 and length == 8:
        candidate_text = ''.join(chars[index:index+length])
        candidate_text = candidate_text.replace('\n','')
        candidates.append((candidate_text,category))
        
    if category == 3 and length > 15:
        candidate_text = ''.join(chars[index:index+length])
        candidate_text = candidate_text.replace('\n','')
        candidates.append((candidate_text,category))
        

In [0]:
# Show predictions

'''
Ignore:           0
Sender Name:      1 
Sender KVK:       2 
Sender IBAN:      3 
Invoice Reference:4
Total:            5
'''

sorted(candidates, key=lambda tup: tup[1])

[('TTN: ING Bank N.V.B', 1), ('DHE8F', 4), ('17817.49', 5)]

In [24]:
print(candidates)

NameError: ignored